In [34]:
import pandas as pd
from graph_tool.all import *
import numpy as np
import matplotlib


In [35]:
from PIL import Image
import os
class PlaceCanvas:
    def __init__(self, width, height):
        self.canvas = Image.new('RGB', (width, height), color = 'white')


    def update_pixel(self, x,y,color):
        """
        Sets the specific x,y coordinate in the canvas to the given hex color.

        example:
        update_pixel(10,10,"#B4FBB8")
        """
        h = color.lstrip('#')
        rgb_value = tuple(int(h[i:i+2], 16) for i in (0, 2, 4))
        self.canvas.putpixel((x,y), rgb_value)

    def save_canvas(self, path = "images/"):
        if(os.path.exists(f'{path}images.png')):
            i = 0 
            while(os.path.exists(f'{path}images {i}.png')):
                i += 1
            self.canvas.save(f'{path}images {i}.png')
        else:
            self.canvas.save(f'{path}images.png')

    def get_image(self):
        return self.canvas

In [36]:
colors = ["#FFFFFF",
"#E4E4E4",
"#888888",
"#222222",
"#FFA7D1",
"#E50000",
"#E59500",
"#A06A42",
"#E5D900",
"#94E044",
"#02BE01",
"#00D3DD",
"#0083C7",
"#0000EA",
"#CF6EE4",
"#820080"]

In [44]:
dict_vertex = {}
dict_pixel = {}
dict_edges = {}

grid_size = 100
g = Graph(directed = False)

# internal properties
v_type = g.new_vertex_property("int") # user id for last action for each pixel
v_filter = g.new_vertex_property("bool") # the color of this action
v_action_count = g.new_vertex_property("int") # action count for the user
v_id = g.new_vertex_property("string") # type of vertex
v_last_action_user = g.new_vertex_property("string") # id for vertex

user_id = 0
canvas_2 = PlaceCanvas(100,100)

with open(filename, "r", newline="") as f:  # on Python 3.x use: open(filename, "r", newline="")
    reader = csv.reader(f)  # create a CSV reader
    header = next(reader)  # grab the first line and keep it as a header reference
    print("CSV header: {}".format(header))
    for row in reader:  # iterate over the available rows
        count += 1
#         print("Processing row: {}".format(row))  # process each row however you want
        date = row[0]
        date = date.replace(' UTC', '')
        try:
            date = datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
        except:
            date = datetime.strptime(date, "%Y-%m-%d %H:%M:%S.%f")
        user = row[1]
        color = row[2]
        coords_raw = row[3].strip().split(',')
        coord0 = int(coords_raw[0])
        coord1 = int(coords_raw[1])
        coords = f"{coord0}|{coord1}"

        
        
        if date.day == 4 and date.hour < 24:
            if coord0 < grid_size and coord1 < grid_size:
                canvas_2.update_pixel(coord0, coord1, color)

                if user not in dict_vertex:
                    # if no user in graph add vertex to dictionary, add vertex to graph
                    dict_vertex[user] = g.add_vertex() # this also adds the vertex to graph g
                    v_id[dict_vertex[user]] = user # internal property of user id for graph vertices
    #                 user_id += 1
                    v_type[dict_vertex[user]] = 0 # internal property of vertex type
                    v_action_count[dict_vertex[user]] = 1 # internal property of user's actions
                else:
                    v_action_count[dict_vertex[user]] += 1 # add action count for the user
                    if v_action_count[dict_vertex[user]] >4: # action count boolean property for filtering out users that did only few actions
                        v_filter[dict_vertex[user]] = True

                if f"{color}|{coords}" not in dict_vertex:
                    # if no color/pixel in graph add vertex to dictionary, add vertex to graph
                    dict_vertex[f"{color}|{coords}"] = g.add_vertex()  
                    v_id[dict_vertex[f"{color}|{coords}"]] = f"{color}|{coords}"
                    v_type[dict_vertex[f"{color}|{coords}"]] = 1
                    v_filter[dict_vertex[f"{color}|{coords}"]] = True # always set to true to not filter out pixels


                if user+'|'+ f"{color}|{coords}" not in dict_edges:
                    # if a user performed exactly the same action twice we want to have it in the dictionary
                    e = g.add_edge(dict_vertex[user], dict_vertex[f"{color}|{coords}"]) # add edge 
                    dict_edges[user+'|'+ f"{color}|{coords}"] = 1 # record that we added this edge already



                v_last_action_user[dict_vertex[f"{color}|{coords}"]] = user # overwrite the last action for a color/pixel by user id
                dict_pixel[coords] = user
        
        else: 
            break

canvas_2.save_canvas(path="")

CSV header: ['timestamp', 'user_id', 'pixel_color', 'coordinate']


In [45]:
g.vp.v_type = v_type
g.vp.v_id = v_id
g.vp.v_last_action_user = v_last_action_user

In [46]:
old_sub = GraphView(g, v_filter)

In [47]:
sub_g = Graph(old_sub, prune=True)

In [48]:
# graph_draw(sub_g, vertex_text=sub_g.vertex_index, output="whole_graph.pdf", vertex_fill_color=sub_g.vp.v_type)

In [49]:
sub_g

<Graph object, undirected, with 4217 vertices and 1194 edges, 3 internal vertex properties, at 0x7f7c9458a320>

# Fillter out unconnected nodes

In [50]:
in_degs = sub_g.get_in_degrees(sub_g.get_vertices())
out_degs = sub_g.get_out_degrees(sub_g.get_vertices())

In [51]:
v_filter_deg = sub_g.new_vertex_property("bool")

In [52]:
for i in range(len(in_degs)):
    if (in_degs[i] != 0) or (out_degs[i] != 0):
        v_filter_deg[i] = True

In [53]:
v_con_sub_g = GraphView(sub_g, v_filter_deg)

In [54]:
v_con_sub_g

<GraphView object, undirected, with 1120 vertices and 1194 edges, 3 internal vertex properties, edges filtered by (<EdgePropertyMap object with value type 'bool', for Graph 0x7f7c94588e50, at 0x7f7c9458bb50>, False), vertices filtered by (<VertexPropertyMap object with value type 'bool', for Graph 0x7f7c94588e50, at 0x7f7c9458ae00>, False), at 0x7f7c94588e50>

In [48]:
# graph_draw(v_con_sub_g, vertex_text=v_con_sub_g.vertex_index, output="filtered_graph.pdf", vertex_fill_color=v_con_sub_g.vp.v_type)

## Communities

In [55]:
state = minimize_blockmodel_dl(v_con_sub_g)

In [58]:
state.draw(output="football-sbm-fit.pdf",vertex_font_size=0.4, vertex_text=v_con_sub_g.vp.v_id)

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7f7c94588e50, at 0x7f7d46995330>

In [56]:
b = state.get_blocks()

In [57]:
v_con_sub_g

<GraphView object, undirected, with 1120 vertices and 1194 edges, 3 internal vertex properties, edges filtered by (<EdgePropertyMap object with value type 'bool', for Graph 0x7f7c94588e50, at 0x7f7c9458bb50>, False), vertices filtered by (<VertexPropertyMap object with value type 'bool', for Graph 0x7f7c94588e50, at 0x7f7c9458ae00>, False), at 0x7f7c94588e50>

In [59]:
df = pd.DataFrame(np.column_stack([list(sub_g.vp.v_id), list(b.a),list(sub_g.vp.v_type)]), columns=['id','community','type'])

In [60]:
df_last = pd.DataFrame(np.column_stack([list(sub_g.vp.v_id), list(sub_g.vp.v_last_action_user),list(sub_g.vp.v_type)]), columns=['id','last', 'type'])

In [61]:
df_last['pixel'] = df_last[df_last.type=="1"].id.str.split('|')

In [62]:
last_pixel = pd.DataFrame(np.column_stack([list(dict_pixel.keys()), list(dict_pixel.values())]), columns = ['coords','user'])

In [63]:
user_community = df[(df.community != 0) & (df.type == '0')]

In [64]:
user_community

,id,community,type
42,9OSX5GxWtkQRnlr59I9o8AMXxF5oFl9lSlu6ifIgQWsGKo...,356,0
47,HUvvfECsO5Hbjequh0mv1qUHzBjn81nv7JMdvTCugBxy55...,927,0
52,CResaAjZPq+4pCEDBDF3aZLV+p0u1dSZwREbRtmgURXWJu...,927,0
60,VLlmc4vEdgKFmK/IUwy24nvYZ9W12KTHwUF7WpNf7dgomn...,356,0
74,3FjdLJ6xHZ9QrlswF0goKO2Huf/F1D7DzC+LqvnyciGtIL...,356,0
...,...,...,...
2533,G/FMnifgQCDe5vtNS7UHDY9ZQQmf+DBm9KvCAfXIzxPbNp...,356,0
2655,ymBoQgSx7KR059cPWUz0G+rAEP2lB+0/kXl2dJ3/4rNCMB...,356,0
2718,24Q0YIJ9QrJwZWDGcGsJhYAVvu9wfYTDFba0Vvuyi3hU/y...,356,0
3357,Mp387agEDK2wAH1FBjSSkkuzP89wLwVoNd6rvtnO5cD8tH...,356,0


In [65]:
df_vis = pd.merge(last_pixel, user_community, how = 'left', left_on='user', right_on = 'id')

In [66]:
df_vis = df_vis.dropna()[['coords','community']]

In [60]:
df_vis.to_csv('df_vis.csv')

In [67]:
df[(df.community != 0) & (df.type == '0')].community.value_counts()

356    151
927     45
Name: community, dtype: int64

## Community vis

In [73]:
canvas = PlaceCanvas(100,100)

In [75]:

colors = dict(zip(df_vis['community'].unique(),
                  (f'{c}' for c in matplotlib.colors.cnames.values())))

In [76]:
df_vis

,coords,community
0,1|1,#F0F8FF
5,67|41,#F0F8FF
48,59|95,#FAEBD7
55,80|84,#F0F8FF
65,12|19,#F0F8FF
...,...,...
2095,49|70,#F0F8FF
2096,72|87,#F0F8FF
2099,48|91,#F0F8FF
2101,68|71,#F0F8FF


In [71]:
df_vis = df_vis.replace({"community": colors})

In [77]:
for index, row in df_vis.iterrows():
    x,y = row.coords.split("|")
    canvas.update_pixel(int(x),int(y),row.community)
    

In [78]:
canvas.save_canvas(path="")

# Tests

In [102]:
v_test = v_con_sub_g.vertex(4275)
v_con_sub_g.vp.v_id[v_test]

'8Zbajz0Scbc0fb2vYNo2BNUnW85I1/TBZPoFUTkODd4z/QRk9/AdvaDYd/F34ZTHkgtk7Nvb4a5hyMJXgfflHQ=='

In [104]:
for key in v_con_sub_g.vp.keys():
    print(key)

v_type
v_id


In [105]:
for edge in g.vertex(4275).all_edges():
    print(edge)

(4275, 4274)


In [115]:
v1 = old_sub.vertex(4275)

In [117]:
old_sub.vp.v_id[v1][:15]

'#00A368"1169|8"'

In [108]:
v1 = sub_g.vertex(4275)

In [118]:
sub_g.vp.v_id[v1][:15]

'8Zbajz0Scbc0fb2'

In [111]:
v1 = g.vertex(8093)

In [112]:
g.vp.v_id[v1]

'8Zbajz0Scbc0fb2vYNo2BNUnW85I1/TBZPoFUTkODd4z/QRk9/AdvaDYd/F34ZTHkgtk7Nvb4a5hyMJXgfflHQ=='

In [21]:
for edge in old_sub.vertex(4275).all_edges():
    print(edge)

In [11]:
sub_g

<Graph object, undirected, with 3994 vertices and 959 edges, 3 internal vertex properties, at 0x7f20a1c87520>

In [22]:
list(sub_g.vp.v_id)

['#D4D7D9"52|7"',
 '1',
 '#FFFFFF"24|96"',
 '#000000"15|2"',
 '#FFFFFF"64|22"',
 '#E4ABFF"0|6"',
 '#515252"54|78"',
 '#7EED56"0|0"',
 '#000000"85|29"',
 '#000000"81|94"',
 '#000000"3|0"',
 '#2450A4"68|98"',
 '11',
 '#00CC78"75|76"',
 '#FFFFFF"2|38"',
 '#000000"52|7"',
 '#FFFFFF"78|11"',
 '#DE107F"4|0"',
 '#000000"69|69"',
 '#000000"34|4"',
 '#FFFFFF"18|87"',
 '#BE0039"93|47"',
 '21',
 '#9C6926"2|65"',
 '#FFFFFF"36|99"',
 '23',
 '#000000"52|64"',
 '#000000"54|65"',
 '#FFFFFF"0|33"',
 '#51E9F4"0|4"',
 '#FFFFFF"5|57"',
 '28',
 '#000000"20|82"',
 '#000000"2|2"',
 '#FFFFFF"60|21"',
 '#E4ABFF"1|81"',
 '#FFFFFF"0|0"',
 '#515252"2|2"',
 '#000000"24|95"',
 '#000000"75|13"',
 '#FFFFFF"26|26"',
 '#000000"33|17"',
 '#FFFFFF"0|1"',
 '#000000"8|28"',
 '#FFFFFF"0|3"',
 '#000000"5|57"',
 '#000000"76|23"',
 '#7EED56"5|0"',
 '#2450A4"69|92"',
 '#000000"80|9"',
 '48',
 '#FFFFFF"21|95"',
 '50',
 '#FFFFFF"31|79"',
 '#000000"56|0"',
 '#811E9F"0|0"',
 '#FFFFFF"74|6"',
 '#FFA800"8|0"',
 '#000000"22|95"',
 '#0

In [15]:
for edge in g.vertex(200).all_edges():
    print(edge)

(200, 199)
(200, 2259)
(200, 3277)
(200, 3280)


In [16]:
for edge in sub_g.vertex(200).all_edges():
    print(edge)

(200, 199)


In [23]:
for edge in v_con_sub_g.vertex(4275).all_edges():
    print(edge)

(4275, 4276)
(4275, 5783)
(4275, 7356)
(4275, 8919)
(4275, 11625)
(4275, 13096)
(4275, 14497)
(4275, 16119)
(4275, 17713)


In [25]:
for edge in dict_vertex['8Zbajz0Scbc0fb2vYNo2BNUnW85I1/TBZPoFUTkODd4z/QRk9/AdvaDYd/F34ZTHkgtk7Nvb4a5hyMJXgfflHQ=='].all_edges():
    print(edge)

(8093, 8094)
(8093, 10976)
(8093, 14092)
(8093, 17252)
(8093, 22818)
(8093, 25944)
(8093, 28964)
(8093, 32542)
(8093, 36025)


In [256]:
dict_vertex

{'neOdfPFywDW+27oZPIQTTND8XrdpxNprL3djflcEse30DvB3cUeXmkk0433s1u6hTKS0sJlauy97nxfffnZD7A==': <Vertex object with index '0' at 0x1d59a07b0>,
 '#FFFFFF"1169|8"': <Vertex object with index '1' at 0x1d59a3370>,
 'Uj516Da76vRFlduIU/9XCjkqKkjkOadNy9l7Uou5kUQT8YLBb8+BUMHb9KhMQ07vOgMdueM4jXrAfOWspzmWFA==': <Vertex object with index '2' at 0x1d59a2340>,
 '#BE0039"9|1349"': <Vertex object with index '3' at 0x1d59a2110>,
 'lSzxKxeOO7wPnHVJZukNkE6KKpPxgfjdFYpTsgcrmsLQXj1M8VcIhxgxzNouuUU+1pgOWIZNwOI3rhZp7U3b0Q==': <Vertex object with index '4' at 0x1d59a2490>,
 '#D4D7D9"52|7"': <Vertex object with index '5' at 0x1d59a1fc0>,
 'NEPi1MYeEzOQ2ectLpueAhcHPH+2qKYLlQWgPBEGrJF1UnEZV8QVKPsZT1SviB4YfQSwUyqfgz0Lp5qnnyH4/A==': <Vertex object with index '6' at 0x1d59a2180>,
 '#FFA800"385|2"': <Vertex object with index '7' at 0x1d59a12a0>,
 '8XKUQZ6w0aHrrZM7Sevh7Fq9QFPl8wLP/XiJxvyzGVed2K9p21zNFd+XvifN6FccpGeHirL1Dsgo0hT1sJ5ZvA==': <Vertex object with index '8' at 0x1d59a1000>,
 '#000000"1|1971"': <Vertex object 